In [ ]:
from keras.datasets import cifar10
import numpy

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('shape of x_train: ' + str(x_train.shape))
print('shape of y_train: ' + str(y_train.shape))
print('shape of x_test: ' + str(x_test.shape))
print('shape of y_test: ' + str(y_test.shape))
print('number of classes: ' + str(numpy.max(y_train) - numpy.min(y_train) + 1))

In [ ]:
def to_one_hot(y, num_class=10):
    one_hot_ret = numpy.zeros((y.shape[0], 10))
    one_hot_ret[numpy.arange((y.shape[0])),y[:,0]] = 1
    return one_hot_ret
    

y_train_vec = to_one_hot(y_train)
y_test_vec = to_one_hot(y_test)

print('Shape of y_train_vec: ' + str(y_train_vec.shape))
print('Shape of y_test_vec: ' + str(y_test_vec.shape))

print(y_train[0])
print(y_train_vec[0])

In [ ]:
rand_indices = numpy.random.permutation(50000)
train_indices = rand_indices[0:40000]
valid_indices = rand_indices[40000:50000]

x_val = x_train[valid_indices, :]
y_val = y_train_vec[valid_indices, :]

x_tr = x_train[train_indices, :]
y_tr = y_train_vec[train_indices, :]

print('Shape of x_tr: ' + str(x_tr.shape))
print('Shape of y_tr: ' + str(y_tr.shape))
print('Shape of x_val: ' + str(x_val.shape))
print('Shape of y_val: ' + str(y_val.shape))

In [ ]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, add, AveragePooling2D, Input
from keras.models import Sequential

def add_bottleneck(X, channel_count, downsample=True, dropout=False):
    X_skip = X
    if True == downsample:
        stride_val = 2
    else:
        stride_val = 1
    if True == dropout:
        X = Dropout(0.3)(X)
    X_out = Conv2D(channel_count, (3, 3), strides = stride_val, padding='same')(X)
    X_out = BatchNormalization()(X_out)
    X_out = Activation('relu')(X_out)
    if True == dropout:
        X_out = Dropout(0.3)(X_out)
    X_out = Conv2D(channel_count, (3, 3), padding='same')(X_out)
    X_out = BatchNormalization()(X_out)
    if True == dropout:
        X_skip = Dropout(0.3)(X_skip)
    X_skip = Conv2D(channel_count, (3, 3), strides = stride_val, padding='same')(X_skip)
    #X_skip = MaxPooling2D((2, 2))(X_skip)
    out = add([X_out, X_skip])
    out = Activation('relu')(out)
    return out
    
input_x = Input(shape=(32, 32, 3))
X = Conv2D(32, (3, 3), padding='same')(input_x)
X = BatchNormalization()(X)
X = Activation('relu')(X)
X = MaxPooling2D((2, 2))(X)
X = add_bottleneck(X, 32, downsample=False)
X = add_bottleneck(X, 32, downsample=False)
X = add_bottleneck(X, 64, downsample=False)
X = add_bottleneck(X, 64, downsample=True)
X = add_bottleneck(X, 128, downsample=False, dropout=False)
X = add_bottleneck(X, 128, downsample=True, dropout=False)
X = add_bottleneck(X, 256, downsample=False, dropout=False)
X = add_bottleneck(X, 256, downsample=False, dropout=False)
X = add_bottleneck(X, 512, downsample=False, dropout=False)
X = add_bottleneck(X, 512, downsample=True, dropout=False)
X = AveragePooling2D((2, 2))(X)
X = Flatten()(X)
X = Dense(10, activation='softmax')(X)
# Create model
model = Model(inputs = input_x, outputs = X, name='ResNet')

model.summary()

In [ ]:
from keras.callbacks import LearningRateScheduler

# learning rate schedule
class step_decay():
    def __init__(self):
        self.initial_lrate = 0.0005
        self.drop = 0.5
        self.epochs_drop = 40.0
        #self.epochs_drop = 5.0
        
    def __call__(self, epoch):
        lrate = self.initial_lrate * numpy.power(self.drop, numpy.floor((1+epoch)/self.epochs_drop))
        print("lrate: ", lrate)
        return lrate

In [ ]:
from keras import optimizers

#learning_rate = 0.0007 # to be tuned!

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0),
              metrics=['acc'])
# learning schedule callback
lrate = LearningRateScheduler(step_decay())
callbacks_list = [lrate]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(
#         rotation_range=45,  
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=5.0,
#         zoom_range=0.5,
#         fill_mode='nearest',
#         horizontal_flip=True,
#         data_format='channels_last',
#         validation_split=0.0)
datagen = ImageDataGenerator(
        rotation_range=40,  
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
        horizontal_flip=True,
        data_format='channels_last',
        validation_split=0.0)
datagen.fit(x_tr)

In [ ]:
history = model.fit_generator(datagen.flow(x_tr, y_tr, batch_size=32), steps_per_epoch=int(len(x_tr)/32), epochs=500, callbacks=callbacks_list, validation_data=(x_val, y_val))